In [3]:
import miniaudio
import numpy as np

In [4]:
# import IPython

In [5]:
def add_vhs_distortion(input_file, output_file, noise_lel):
    # Decode the input WAV file
    audio_data = miniaudio.decode_file(input_file)

    # Apply VHS distortion effect
    noise_level = noise_lel  # Adjust noise level for distortion
    num_samples = len(audio_data.samples)

    # Create noise to add to the audio signal
    noise = np.random.normal(0, noise_level, num_samples).astype(np.float32)

    # Combine original audio with noise
    distorted_samples = audio_data.samples + noise

    # Ensure samples are within valid range
    distorted_samples = np.clip(distorted_samples, -1.0, 1.0)

    # Convert distorted samples back to int16 format for WAV writing
    distorted_samples_int16 = (distorted_samples * 32767).astype(np.int16)
    distorted_sound = miniaudio.DecodedSoundFile('distortion', audio_data.nchannels, audio_data.sample_rate, miniaudio.SampleFormat.SIGNED16, distorted_samples_int16)
    miniaudio.wav_write_file(output_file, distorted_sound)
    # Write the distorted audio to a new WAV file
    # miniaudio.wav_write_file(output_file, distorted_samples_int16.tobytes(), 
    #                      sample_rate=audio_data.sample_rate, 
    #                      nchannels=audio_data.nchannels,  # Use nchannels instead of channels
    #                      sample_format=miniaudio.SampleFormat.SIGNED16)

In [6]:
def load_wav(file_path):
    # Load the WAV file using miniaudio
    with open(file_path, 'rb') as f:
        wav = miniaudio.decode(f.read())
    
    return wav

def save_wav(file_path, samples, sample_rate, num_channels):
    # num_channels = 1;
    # Save the processed samples back to a WAV file
    wav_data = np.int16(samples * 32767).astype(np.int16)#.tobytes()
    distorted_sound = miniaudio.DecodedSoundFile('distortion', num_channels, sample_rate, miniaudio.SampleFormat.SIGNED16, wav_data)
    miniaudio.wav_write_file(file_path, distorted_sound)

In [7]:
def apply_vhs_effect(samples, sample_rate, noise_level):
    normalized_samples = samples / np.max(np.abs(samples))
    # Add some noise to create distortion
    noise = np.random.normal(0, noise_level, len(normalized_samples))
    distorted_samples = normalized_samples + noise

    # Simple low-pass filter to simulate VHS tape characteristics
    # Using a simple moving average for demonstration
    window_size = int(sample_rate * 0.1)  # 100ms window
    filtered_samples = np.convolve(distorted_samples, np.ones(window_size)/window_size, mode='same')

    # Scale back to original range
    final_samples = np.clip(filtered_samples * np.max(np.abs(samples)), -1, 1)

    return final_samples

In [14]:
def apply_vhs_effect_2(samples, sample_rate, num_channels, noise_level):
    print(f"len of samples:{len(samples)}")
    t = np.arange(len(samples)) / sample_rate
    wow = (np.sin(2 * np.pi * 0.5 * t) * 0.001)
    flutter = (np.sin(2 * np.pi * 10 * t) * 0.0001)
    time_variation = 1 + wow + flutter
    print(f"len of time_variation:{len(time_variation)}")
    new_t = (np.cumsum(time_variation) / sample_rate)
    print(f"len of new_t:{len(new_t)}")
    samples_wow_flutter = np.array([np.interp(t, new_t, samples[:, i]) for i in range(num_channels)]).T
    # samples_wow_flutter = np.array([np.interp(t, new_t, samples[:])]).T
    # for i in range(len(samples)):
    #     samples[i] *= time_variation[i]
    # samples = samples.T;

    # Clip the audio to prevent overflow
    # samples = np.clip(samples, -1, 1)
    # print(f"shape of sample_wow_flitter:{samples_wow_flutter.shape}")
    # print(f"samples shape:{samples.shape}")
    
    # return samples
    # # Add noise
    # noise = np.random.normal(0, noise_level, samples.shape).astype(np.float16)
    # print(f"noise shape:{noise.shape}")
    # samples_with_noise = samples_wow_flutter + noise
    noise = np.random.normal(0, 0.001, len(samples_wow_flutter)).astype(np.float16)
    print(f"shape of noise:{noise.shape}")
    for j in range(len(samples_wow_flutter)):
        samples_wow_flutter[j] += noise[j]
    # samples_with_noise = samples_wow_flutter + noise
    # # Add dropouts
    dropout_mask = np.random.random(len(samples)) > 0.001
    samples_with_dropouts = samples_wow_flutter * dropout_mask[:, np.newaxis]

    # # Clip the audio to prevent overflow
    samples_with_dropouts = np.clip(samples_with_dropouts, -1, 1)

    return samples_with_dropouts

In [15]:
def run_apply_vhs_effect(input_file, output_file, noise_level):
    # Load the WAV file
    wav = load_wav(input_file)
    # Convert byte data to numpy array
    samples = np.frombuffer(wav.samples, dtype=np.int16)
    # Apply VHS effect
    distorted_samples = apply_vhs_effect_2(samples, wav.sample_rate, 1, noise_level)
    # Save the new WAV file
    save_wav(output_file, distorted_samples, wav.sample_rate, wav.nchannels)

In [16]:
# add_vhs_distortion("test01.wav", "output_distorted_08.wav", 0.01);
run_apply_vhs_effect("test02.wav", "output_distorted.wav", 0.001)

len of samples:743036
len of time_variation:743036
len of new_t:743036
shape of noise:(743036,)


### The following codes are not used

In [ ]:
# import miniaudio
# import numpy as np
# import random

# def add_vhs_distortion(input_file, output_file):
#     # Read the input file
#     decoder = miniaudio.decode_file(input_file)
#     samples = decoder.samples
#     sample_rate = decoder.sample_rate
#     num_channels = decoder.nchannels

#     # Convert to float32 for processing
#     samples = samples.astype(np.float32)

#     # Reshape the samples array to separate channels
#     samples = samples.reshape(-1, num_channels)

#     # Add wow and flutter effect
#     t = np.arange(len(samples)) / sample_rate
#     wow = np.sin(2 * np.pi * 0.5 * t) * 0.01
#     flutter = np.sin(2 * np.pi * 10 * t) * 0.001
#     time_variation = 1 + wow + flutter
#     new_t = np.cumsum(time_variation) / sample_rate
#     samples_wow_flutter = np.array([np.interp(t, new_t, samples[:, i]) for i in range(num_channels)]).T

#     # Add noise
#     noise = np.random.normal(0, 0.005, samples.shape)
#     samples_with_noise = samples_wow_flutter + noise

#     # Add dropouts
#     dropout_mask = np.random.random(len(samples)) > 0.001
#     samples_with_dropouts = samples_with_noise * dropout_mask[:, np.newaxis]

#     # Clip the audio to prevent overflow
#     samples_with_dropouts = np.clip(samples_with_dropouts, -1, 1)

#     # Convert back to int16
#     distorted_samples = (samples_with_dropouts * 32767).astype(np.int16)

#     # Flatten the array
#     distorted_samples = distorted_samples.flatten()

#     # Write the output file
#     with miniaudio.WAVWriter(output_file, sample_rate, num_channels) as writer:
#         writer.write_pcm_frames(distorted_samples)

# # Usage
# input_file = "input.wav"
# output_file = "output_vhs_distorted.wav"
# add_vhs_distortion(input_file, output_file)